In [1]:
%pylab inline
import os
import glob
import pandas as pd
import re
from collections import OrderedDict
import seaborn as sns
sns.set_context('paper', font_scale=2)
sns.set_style('white')
def clean_tx(tx):
    return re.sub(r'\.[0-9]+', '', tx)

root_dir = '/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679/ribocop_results_Feb2019_longest/'

def get_uorf(df):
    return df.loc[df.ORF_type=='uORF',]
def get_translating(df):
    return df.loc[df.status=='translating']
def get_tx_from_orfid(orfid):
    splitted = orfid.split('_')
    if len(splitted) == 4:
        txid = splitted[0]
    elif len(splitted)==5:
        txid = splitted[0] + '_' + splitted[1]
    else:
        raise RuntimeError('Found:{}'.format(splitted))
    return txid

Populating the interactive namespace from numpy and matplotlib


In [4]:
annotation = pd.read_table('/staging/as/skchoudh/ribocop_hg38_feb2019_annotation_longest_candidate_orfs.tsv').set_index('ORF_ID')
annotation

,ORF_type,transcript_id,transcript_type,gene_id,gene_name,gene_type,chrom,strand,start_codon,coordinate
ORF_ID,,,,,,,,,,
ENST00000335137.3_69091_70005_915,annotated,ENST00000335137.3,protein_coding,ENSG00000186092.4,OR4F5,protein_coding,chr1,+,ATG,69091-70005
ENST00000624431.1_182709_184155_399,annotated,ENST00000624431.1,protein_coding,ENSG00000279928.1,FO538757.2,protein_coding,chr1,+,ATG,"182709-182746,183114-183240,183922-184155"
ENST00000623834.3_185220_195411_1077,annotated,ENST00000623834.3,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"185220-185350,185491-185559,187376-187577,1877..."
ENST00000623083.3_185220_195411_1392,annotated,ENST00000623083.3,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"185220-185350,185491-185559,186317-186469,1871..."
ENST00000624735.1_184930_200086_1365,annotated,ENST00000624735.1,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"184930-184971,184977-185049,185529-185559,1863..."
ENST00000426406.2_450743_451678_936,annotated,ENST00000426406.2,protein_coding,ENSG00000278566.1,OR4F29,protein_coding,chr1,-,ATG,450743-451678
ENST00000332831.3_685719_686654_936,annotated,ENST00000332831.3,protein_coding,ENSG00000273547.1,OR4F16,protein_coding,chr1,-,ATG,685719-686654
ENST00000420190.6_924432_939291_1074,annotated,ENST00000420190.6,protein_coding,ENSG00000187634.11,SAMD11,protein_coding,chr1,+,ATG,"924432-924948,925922-926013,930155-930336,9310..."
ENST00000437963.5_925942_935793_327,annotated,ENST00000437963.5,protein_coding,ENSG00000187634.11,SAMD11,protein_coding,chr1,+,ATG,"925942-926013,930155-930336,931039-931089,9357..."


In [5]:
orf_df = OrderedDict()
for f in glob.glob('{}/*_translating_ORFs.tsv'.format(root_dir)):
    filename = os.path.basename(f).replace('_translating_ORFs.tsv', '')
    orf_df[filename] = pd.read_table(f)


In [6]:
orf_df_annotated = OrderedDict()
for key, df in orf_df.items():
    df = df[df.ORF_type=='annotated'].sort_values(by=['transcript_id', 'ORF_ID', 'gene_id'])
    orf_df_annotated[key] = df
    print(df.shape)

(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)
(94421, 16)


In [7]:
orf_df_annotated[key] 

,ORF_ID,ORF_type,status,phase_score,read_count,length,valid_codons,transcript_id,transcript_type,gene_id,gene_name,gene_type,chrom,strand,start_codon,profile
34914,ENST00000000233.9_127588499_127591296_540,annotated,nontranslating,0.202265,192,540,106,ENST00000000233.9,protein_coding,ENSG00000004059.10,ARF5,protein_coding,chr7,+,ATG,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
52314,ENST00000000412.7_8941821_8946404_831,annotated,nontranslating,0.091473,84,831,67,ENST00000000412.7,protein_coding,ENSG00000003056.7,M6PR,protein_coding,chr12,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ..."
48429,ENST00000000442.10_64307180_64315963_1269,annotated,nontranslating,0.084490,51,1269,41,ENST00000000442.10,protein_coding,ENSG00000173153.13,ESRRA,protein_coding,chr11,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
51770,ENST00000001008.5_2795140_2803255_1377,annotated,nontranslating,0.042876,209,1377,142,ENST00000001008.5,protein_coding,ENSG00000004478.7,FKBP4,protein_coding,chr12,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ..."
9599,ENST00000001146.6_72132230_72147834_1536,annotated,nontranslating,0.000000,0,1536,0,ENST00000001146.6,protein_coding,ENSG00000003137.8,CYP26B1,protein_coding,chr2,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8746,ENST00000002125.8_37231706_37248347_1323,annotated,nontranslating,0.334053,38,1323,34,ENST00000002125.8,protein_coding,ENSG00000003509.15,NDUFAF7,protein_coding,chr2,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
31055,ENST00000002165.10_143495710_143511634_1401,annotated,nontranslating,0.081983,206,1401,151,ENST00000002165.10,protein_coding,ENSG00000001036.13,FUCA2,protein_coding,chr6,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
70646,ENST00000002501.10_90006338_90019341_474,annotated,nontranslating,0.188982,14,474,14,ENST00000002501.10,protein_coding,ENSG00000003249.13,DBNDD1,protein_coding,chr16,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
20284,ENST00000002596.5_11399085_11400005_921,annotated,nontranslating,1.000000,2,921,2,ENST00000002596.5,protein_coding,ENSG00000002587.9,HS3ST1,protein_coding,chr4,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15911,ENST00000002829.7_50159623_50188112_2355,annotated,nontranslating,0.098748,237,2355,191,ENST00000002829.7,protein_coding,ENSG00000001617.11,SEMA3F,protein_coding,chr3,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
def eval_profile(profile):
    profile = eval(profile)
    return sum(profile)



In [ ]:
orf_profiles_sum = OrderedDict()
for key, df in orf_df_annotated.items():
    df['profile_sum'] = df.profile.apply(eval_profile)
    orf_profiles_sum[key] = df[['transcript_id', 'profile_sum']]


In [ ]:
for key,df in orf_profiles_sum.items():
    df.to_csv('/staging/as/skchoudh/SRP010679_tx_counts/{}.tsv'.format(key), sep='\t', header=None, index=False)
    

In [ ]:
def get_start_end_coordinates(orf_id):
    #orf_id = row['ORF_ID']
    splitted = orf_id.split('_')
    stop = splitted[-2]
    start = splitted[-3]
    return pd.Series([int(start), int(stop)])


# Approach

We need the transcript level counts to process in riborex. We take the simple approach of first groupring 
everythong at the gene level. We only care about the protein_coding genes and the annotated ones.

In [ ]:
selected_df = orf_df[filename]

selected_df[['start', 'stop']] =selected_df['ORF_ID'].apply(get_start_end_coordinates)
selected_df_grouped = selected_df.groupby(['gene_id', 'transcript_id'])

    
    


In [ ]:
for key, group_df in selected_df_grouped:
    profile_series_sum = pd.Series([])
    index_series_sum = pd.Series([])
    for index, row in group_df.iterrows():
        profile = eval(row['profile'])
        profile_series = pd.Series(profile, index=range(int(row['start']), int(row['stop'])+1 ))
        index_series = pd.Series([1]*(row['stop']-row['start']+1),  index=range(int(row['start']), int(row['stop'])+1 ))
        profile_series_sum = profile_series_sum.add(profile_series)
        index_series_sum = index_series_sum.add(index_series)
    break
    

In [ ]:
orf_id_range_index = {}
for orf_id, row in annotation.iterrows():
    intervals_string = row['coordinate']
    intervals = intervals_string.split(',')
    interval_range_index = []
    for interval in intervals:
        start, end = interval.split('-')
        interval_range_index = interval_range_index + list(range(int(start), int(end)+1))
    orf_id_range_index[orf_id] = interval_range_index